# LightFM table features
UUIDとともにテーブル特徴を取り込む

In [1]:
import datetime

import faiss
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import psutil
from lightfm import LightFM

import schema
from metric import mapk
from scipy import sparse

In [2]:
articles = pd.read_csv('input/transformed/articles.csv', usecols=schema.ARTICLES.keys(), dtype=schema.ARTICLES)
customers = pd.read_csv('input/transformed/customers.csv', usecols=schema.CUSTOMERS.keys(), dtype=schema.CUSTOMERS)
n_user = len(customers)
n_item = len(articles)

In [3]:
transactions = pd.read_csv('input/transformed/transactions_train.csv', parse_dates=['t_dat'], usecols=list(schema.TRANSACTIONS.keys())+['t_dat'], dtype=schema.TRANSACTIONS)
TOPK = 12

In [13]:
def create_customer_features(customers):
    df = customers.copy().drop('customer_id_idx', axis=1)
    df['age_is_null'] = df['age'].isnull().astype(int)
    df['age'] = df['age'].fillna(df['age'].mean())
    for c in ['club_member_status_idx', 'fashion_news_frequency_idx']:
        df = pd.concat([df, pd.get_dummies(df[c], prefix=c)], axis=1).drop(c, axis=1)

    idxs = sparse.identity(len(df), dtype='f')
    return sparse.hstack([idxs, df.values]).astype('float32')


def create_article_features(articles):
    df = articles.copy().drop('article_id_idx', axis=1)
    for c in df.columns:
        df = pd.concat([df, pd.get_dummies(df[c], prefix=c)], axis=1).drop(c, axis=1)

    idxs = sparse.identity(len(df), dtype='f')
    return sparse.hstack([idxs, df.values]).astype('float32')

user_features = create_customer_features(customers)
item_features = create_article_features(articles)

In [20]:
def objective(trial: optuna.Trial) -> float:
    train_days = trial.suggest_int('train_days', 7, 28, 7)
    no_components = trial.suggest_int('no_components', 128, 1024, 128)
    learning_schedule = 'adadelta'
    loss = 'bpr'
    learning_rate = trial.suggest_loguniform('learning_rate', 5*1e-3, 5*1e-2)
    item_alpha = trial.suggest_loguniform('item_alpha', 1e-12, 1e-6)
    user_alpha = trial.suggest_loguniform('user_alpha', 1e-12, 1e-6)
    max_sampled = trial.suggest_int('max_sampled', 10, 20)
    epochs = 20

    lightfm_params = {
        'no_components': no_components,
        'learning_schedule': learning_schedule,
        'loss': loss,
        'learning_rate': learning_rate,
        'item_alpha': item_alpha,
        'user_alpha': user_alpha,
        'max_sampled': max_sampled,
    }
    print(lightfm_params)

    valid_start_date = datetime.date(2020, 9, 16)
    valid_end_date = datetime.date(2020, 9, 22)
    train_end_date = valid_start_date - datetime.timedelta(days=1)
    train_start_date = valid_start_date - datetime.timedelta(days=train_days)

    transactions_train = transactions.query("@train_start_date <= t_dat <= @train_end_date")
    transactions_valid = transactions.query("@valid_start_date <= t_dat <= @valid_end_date")

    val = transactions_valid.groupby('customer_id_idx')['article_id_idx'].apply(list).reset_index()

    train = sparse.lil_matrix((n_user, n_item))
    train[transactions_train.customer_id_idx, transactions_train.article_id_idx] = 1

    model = LightFM(**lightfm_params)
    model.fit(train, user_features=user_features, item_features=item_features, epochs=epochs, num_threads=psutil.cpu_count(logical=False), verbose=True)

    index = faiss.index_factory(no_components, "Flat", faiss.METRIC_INNER_PRODUCT)
    index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, index)
    index.add(model.item_embeddings)
    _, idxs = index.search(model.user_embeddings, TOPK)

    return mapk(val.article_id_idx, idxs[val.customer_id_idx])

In [21]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=10800)

[I 2022-03-04 18:45:37,374] A new study created in memory with name: no-name-4ed06eab-d012-4f11-83d8-47aa904aaad4


{'no_components': 384, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.04438449855265177, 'item_alpha': 1.424971069814322e-12, 'user_alpha': 7.225127522877115e-09, 'max_sampled': 12}


Epoch: 100%|██████████| 20/20 [08:35<00:00, 25.80s/it]
[I 2022-03-04 18:54:39,796] Trial 0 finished with value: 0.01143350442683491 and parameters: {'train_days': 28, 'no_components': 384, 'learning_rate': 0.04438449855265177, 'item_alpha': 1.424971069814322e-12, 'user_alpha': 7.225127522877115e-09, 'max_sampled': 12}. Best is trial 0 with value: 0.01143350442683491.


{'no_components': 768, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.013067997329139045, 'item_alpha': 1.5731211546179455e-12, 'user_alpha': 2.9335066353527017e-08, 'max_sampled': 19}


Epoch: 100%|██████████| 20/20 [14:07<00:00, 42.40s/it]
[I 2022-03-04 19:09:32,392] Trial 1 finished with value: 0.012845120156728552 and parameters: {'train_days': 28, 'no_components': 768, 'learning_rate': 0.013067997329139045, 'item_alpha': 1.5731211546179455e-12, 'user_alpha': 2.9335066353527017e-08, 'max_sampled': 19}. Best is trial 1 with value: 0.012845120156728552.


...

Epoch: 100%|██████████| 20/20 [16:18<00:00, 48.92s/it]
[I 2022-03-04 21:28:15,150] Trial 16 finished with value: 0.013132215308796704 and parameters: {'train_days': 28, 'no_components': 1024, 'learning_rate': 0.03509733554285025, 'item_alpha': 2.548398644343316e-10, 'user_alpha': 8.409130976927917e-07, 'max_sampled': 16}. Best is trial 16 with value: 0.013132215308796704.


{'no_components': 1024, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.038332971013971893, 'item_alpha': 1.883294566140812e-11, 'user_alpha': 9.250226238062989e-07, 'max_sampled': 20}


Epoch: 100%|██████████| 20/20 [13:26<00:00, 40.33s/it]
[I 2022-03-04 21:42:33,243] Trial 17 finished with value: 0.01245508104203842 and parameters: {'train_days': 21, 'no_components': 1024, 'learning_rate': 0.038332971013971893, 'item_alpha': 1.883294566140812e-11, 'user_alpha': 9.250226238062989e-07, 'max_sampled': 20}. Best is trial 16 with value: 0.013132215308796704.


{'no_components': 896, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.008053964283859993, 'item_alpha': 2.1425655653289647e-10, 'user_alpha': 2.483083625435052e-07, 'max_sampled': 17}


Epoch: 100%|██████████| 20/20 [16:36<00:00, 49.81s/it]
[I 2022-03-04 21:59:59,621] Trial 18 finished with value: 0.012710893740753917 and parameters: {'train_days': 28, 'no_components': 896, 'learning_rate': 0.008053964283859993, 'item_alpha': 2.1425655653289647e-10, 'user_alpha': 2.483083625435052e-07, 'max_sampled': 17}. Best is trial 16 with value: 0.013132215308796704.


In [22]:
study.trials_dataframe().sort_values(by='value', ascending=False).head(20)

,number,value,datetime_start,datetime_complete,duration,params_item_alpha,params_learning_rate,params_max_sampled,params_no_components,params_train_days,params_user_alpha,state
16,16,0.013132,2022-03-04 21:11:05.598831,2022-03-04 21:28:15.149825,0 days 00:17:09.550994,2.548399e-10,0.035097,16,1024,28,8.409131e-07,COMPLETE
1,1,0.012845,2022-03-04 18:54:39.797119,2022-03-04 19:09:32.392005,0 days 00:14:52.594886,1.573121e-12,0.013068,19,768,28,2.933507e-08,COMPLETE
18,18,0.012711,2022-03-04 21:42:33.245008,2022-03-04 21:59:59.621193,0 days 00:17:26.376185,2.142566e-10,0.008054,17,896,28,2.483084e-07,COMPLETE
11,11,0.012666,2022-03-04 20:11:20.499422,2022-03-04 20:26:36.528212,0 days 00:15:16.028790,2.446532e-08,0.025371,15,768,28,2.909309e-11,COMPLETE
13,13,0.012572,2022-03-04 20:38:31.685981,2022-03-04 20:55:49.120267,0 days 00:17:17.434286,3.863379e-08,0.008951,14,896,28,3.462294e-10,COMPLETE
17,17,0.012455,2022-03-04 21:28:15.150985,2022-03-04 21:42:33.243713,0 days 00:14:18.092728,1.883295e-11,0.038333,20,1024,21,9.250226e-07,COMPLETE
12,12,0.012442,2022-03-04 20:26:36.529609,2022-03-04 20:38:31.684754,0 days 00:11:55.155145,9.381584e-11,0.031417,16,768,21,1.168503e-12,COMPLETE
9,9,0.012393,2022-03-04 19:52:18.231092,2022-03-04 20:06:09.758676,0 days 00:13:51.527584,1.433060e-08,0.022381,14,640,28,4.305588e-11,COMPLETE
14,14,0.011891,2022-03-04 20:55:49.121503,2022-03-04 21:06:29.438814,0 days 00:10:40.317311,2.793393e-07,0.024338,17,640,21,2.645919e-09,COMPLETE
0,0,0.011434,2022-03-04 18:45:37.377221,2022-03-04 18:54:39.795831,0 days 00:09:02.418610,1.424971e-12,0.044384,12,384,28,7.225128e-09,COMPLETE


この実験では特徴いれないほうがよい